In [1]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#import graphviz
import numpy as np
import pandas as pd
import xgboost as xgb
import sys
import kaggle_xgb
import calc_ndcg
#from sklearn.grid_search import ParameterGrid

Read in just the testing and training data.

In [14]:
#train_subset = pd.read_csv('../data/train_users_2.csv', nrows = 100000, index_col='id')
#train_data = pd.read_csv('../data/train_users_2.csv', index_col = 'id')
#test_data = pd.read_csv('../data/test_users.csv', index_col = 'id')
sessions = pd.read_csv('../data/sessions.csv')
#age = pd.read_csv('../data/age_gender_bkts.csv')
#countries = pd.read_csv('../data/countries.csv')

all_df is a pickled obect where the training and test has already been concatenated onall columns except 'country_destination'

In [15]:
sessions['secs_elapsed'] = sessions['secs_elapsed']/(3600)

In [16]:
sessions

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,0.088611
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,18.820278
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,0.083611
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,6.150278
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,0.120833
5,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,2.139722
6,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,0.031944
7,d1mm9tcy42,personalize,data,wishlist_content_update,Windows Desktop,0.230833
8,d1mm9tcy42,index,view,view_search_results,Windows Desktop,5.789444
9,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,0.189722


In [21]:
sessions.dropna(subset=['user_id'], inplace = True)

sessions_tr = sessions[sessions.user_id.isin(train_data.index)]
sessions_test = sessions[sessions.user_id.isin(test_data.index)]
a_ignore = set(sessions_tr.action.unique()) ^ set(sessions_test.action.unique())
#sessions.groupby('action')['action_detail']

#set(sessions_test.action_detail.unique()) ^ set(sessions_tr.action_detail.unique())
#sessions_test.action_detail.unique()
#sessions_tr.action_detail.unique()
#train_data['date_account_created'] = pd.to_datetime(train_data.date_account_created)
#x = train_data[train_data.date_account_created>='2014']
#train_data[train_data.index.isin(sessions_tr.user_id)].timestamp_first_active

In [ ]:
actions50 = [sessions_test.groupby('action')['user_id'].nunique() >50]
print 'Number of unique actions with more than 50 users:', np.sum(sessions_test.groupby('action')['user_id'].nunique()>50)

In [ ]:
#Let's get the actions with just one action type and one action detail.
print 'Number of unique actions with just one action type:',
np.sum(sessions.groupby('action')['action_detail'].nunique()==1)
#Take the intersection of these actions with a_ignore.

In [22]:
sessions['action_type'] = sessions['action_type'].fillna('missing')
sessions['action_detail'] = sessions['action_detail'].fillna('missing')

In [ ]:
#actions with null values
actions_null2 = sessions[pd.isnull(sessions.action_type)].action.unique()
actions_null == actions_null2

In [6]:
df = pd.DataFrame({'A': ['foo','bar','foo','foo', 'zoo','bar','foo','foo'],
                       'B': ['coo', np.NaN,'coo','coo', 'dude', np.NaN,'fooo','fooo']})

In [16]:
def test(x):
    print x.B.unqiue
    return x.B.nunique() == 1
df.groupby('A').filter(lambda x: test(x))

,A,B
4,zoo,dude


In [23]:
def custom_filter(x):
     return (x['action_detail'].nunique() == 1) and (x['action_type'].nunique() == 1) 
#get all actions with just one action_type and action 
actions_df = sessions.groupby('action').filter(lambda x: custom_filter(x))

In [24]:
#exclude those actions that are found in one table or the other
actions_df = actions_df[~actions_df.action.isin(a_ignore)]

In [32]:
actions_df.action[pd.isnull(actions_df['secs_elapsed'])].value_counts()

active                         11122
callback                        9029
impressions                     1112
header_userpic                   488
facebook_auto_login              344
dashboard                        135
at_checkpoint                     42
personalize                       37
set_user                          20
ask_question                      20
signup_login                      16
search_results                    12
manage_listing                    11
message_to_host_focus              7
edit_verification                  6
cancellation_policies              6
account                            4
ajax_refresh_subtotal              4
qt2                                3
top_destinations                   3
travel_plans_current               3
similar_listings                   3
payment_methods                    2
settings                           2
lookup                             2
ajax_lwlb_contact                  2
verify                             2
a

In [25]:
#info = actions_1.groupby(['user_id', 'action']).apply(lambda x: x['secs_elapsed'].sum())
action_cnts = actions_df.groupby(['user_id', 'action']).apply(lambda x: x['device_type'].count())

In [26]:
action_cnts = action_cnts.unstack(level=-1)

In [27]:
action_cnts.fillna(0, inplace=True)

In [28]:
pd.to_pickle(action_cnts, '../data/actions.p')

In [ ]:
#features with no time
#null_time, cnts = np.unique(actions_1[pd.isnull(actions_1['secs_elapsed'])].action, return_counts= True)
#zip(cnts, null_time)

In [ ]:
#find the number of users in test data with an action that is not found in the training sessions data
#We should look to find similar items to substitute them
for i in a_ignore:
    users = sessions_test[sessions_test.action == i].user_id.unique()
    if len(users) > 0:
        print i, len(users)

In [19]:
actions_df.action_type.unique()

array(['click', 'data', 'submit', '-unknown-', 'view', 'message_post',
       'partner_callback', 'booking_request', nan, 'booking_response'], dtype=object)

In [ ]:
#iterate through each action and find those that have more than one action detail/ action type
#s = sessions.groupby('action')['action_type'].nunique()
#sessions.groupby('action')['action_detail'].nunique()

In [ ]:
#convert targets to numerical values
targets = train_subset['country_destination']
targets_le = preprocessing.LabelEncoder()
targets = targets_le.fit_transform(targets)

In [ ]:
np.unique(targets)

In [ ]:
#pd.DataFrame(columns=['row_above', 'row_below', ])
results.sort_values(by= 'test-error-mean')

In [ ]:
def ndcg(preds, labels):
    """Calculate sum of normalzied discounted cumulative gain for the predictions
    The correct prediction will have a relevance of 1, incorrect predictions will have a relevance of 0.
    Weight the relevance values such that it is reduced logarithmically proportional to it its position     
    
    Args:
        preds: n*5 array of predictor targets
        labels: n*1 array targets
    Returns:
        sum of normalized discounted cumulative gain for all the predictions
    """
    
    #find positions where the prediction matches the label
    relv_pos = np.where(np.equal(preds, labels))[1]
    #weight func: log2(i+1), add one more to adjust for 0-based indexing
    total_ndcg = np.sum(1./np.log2(relv_pos+2))
    return total_ndcg
    
def evalerror(cls_prob, dtrain):
    """find top k predictions from probability matrix and call ndcg to find accuracy of predictions
    
    Args:
        cls_prob: 2D array, probability of each class for each person (n persons by m classes),
                  the column index corresponds with class
        labels: labels for the n persons
    returns:
        prediction accuracy using ndcg to evaluate predictions of each AirBNB user
    """
    #determine the top k predictions
    labels = dtrain.get_label()
    k = 5
    top_k = cls_prob.argsort(axis = 1)[:,:k:-1]
    #convert true values  and compared with predictions to check for equality
    labels = labels[:, None]
    return 'error', 1.-ndcg(top_k, labels)/len(labels)

In [ ]:
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, all_df.columns[int(feat[1:])], score) for feat, score in features]